In [3]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 7.9 MB/s 
     |████████████████████████████████| 6.6 MB 36.9 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 880 kB 51.3 MB/s 
     |████████████████████████████████| 596 kB 44.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7d31a77c94de6b276a1da6d67493fce0e522e385b66770d3f79484ee3fe7156f
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
from tqdm import tqdm
import csv
import sys
import os

In [2]:
tokenizer=BertTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Similarity')
model=BertForSequenceClassification.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Similarity')

In [3]:
texta='今天的饭不好吃'
textb='今天心情不好'

output=model(torch.tensor([tokenizer.encode(texta,textb)]))
torch.nn.functional.softmax(output.logits,dim=-1)[0][0]
# 左：不相似概率
# 右：相似概率

tensor(0.9998, grad_fn=<SelectBackward0>)

In [4]:
corpus_path = ["data/bq_corpus/test.tsv", "data/lcqmc/test.tsv", "data/paws-x-zh/test.tsv"]
for path in corpus_path[:1]:
    cls_result = []
    test_pairs = open(path, encoding='utf-8').readlines() 
    
    for pair in tqdm(test_pairs):
        pair = pair.split('\t')
        
        output=model(torch.tensor([tokenizer.encode(pair[0],pair[1])]))
        if torch.nn.functional.softmax(output.logits,dim=-1)[0][1] > 0.5:
            cls_result.append(1)
        else:
            cls_result.append(0)
              
    with open('result/' + path.split('/')[1] + '.tsv', 'w') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(['index', 'prediction'])
        for i in range(len(cls_result)):
            tsv_writer.writerow([str(i), str(cls_result[i])])

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [12:13<00:00, 13.64it/s]
